 <div>
 <h1><center>Applications de l'Informatique Quantique</h1>
<h2><center>Séance 1, Notebook 2 : Premier algorithme simple : la recherche de Grover</h2>
<h3> Professeur: Durantin Gautier</h3>
</div>
    
    

### Objectifs du notebook
    
Ce notebook vise à implémenter concrètement un premier algorithme simple, en l'occurrence une recherche de Grover. Il vise à **appréhender la manière de coder et d'exécuter un algorithme quantique, et de comprendre l'impact du bruit sur ces algorithmes**. 

### Rappels sur l'algorithme de Grover

L'algorithme de Grover permet la recherche d'un ou plusieurs éléments répondant à un critère en une complexité $O(\sqrt{N})$ (contre $O(N)$ pour l'approche classique).

Elle se compose des trois étapes suivantes :$⟨\psi|A|\psi⟩$
- Préparation d'un état de superposition uniforme, noté $|s⟩$
- Application de l'oracle $U=I-2|\omega⟩⟨\omega|$, où $|\omega⟩$ est l'état marqué correspondant à l'élément à rechercher. Cet oracle renverse la phase de l'état marqué et préserve tous les autres.
- Application de l'amplificateur d'amplitude $U=2|s⟩⟨s|-I$. Cet état inverse la phase de tous les états sauf celui de superposition uniforme. Dans la pratique, il amplifie le déphasage réalisé sur l'état $|\omega⟩$ et le rend plus probable.

Les deux dernières étapes sont répétées $r=\frac{\pi}{4.arcsin(\frac{1}{\sqrt{2^n}})}$ fois pour maximiser la probabilité de mesurer l'état $|\omega⟩$ en sortie

### Imports
    
Le notebook, comme la plupart des applications qui seront décrites dans ce module, s'appuie sur deux librairies principales :
- *qiskit* : pour la définition des circuits quantiques et leur préparation pour l'exécution
- *qiskit_aer* : pour la simulation du comportement des ordinateurs quantiques. Le module nous permettra d'exécuter nos circuits en pratique

In [ ]:
from qiskit.circuit import QuantumCircuit
from qiskit_aer.primitives import SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.visualization import plot_histogram

from qiskit.circuit.library import grover_operator

from qiskit_aer import AerSimulator

import math

### 1. Exemple pratique

Pour cet exemple pratique, on cherche à marquer l'état $|10⟩$

#### 1.1. Préparation de l'oracle

In [ ]:
oracle=QuantumCircuit(2)

oracle.x(0)
oracle.cz(0,1)
oracle.x(0)


oracle.draw()

#### 1.2. Définition de l'opérateur de Grover

L'opérateur de Grover peut soit être défini manuellement :

In [ ]:
amplif_amplit = QuantumCircuit(2)
amplif_amplit.h([0,1])
amplif_amplit.x([0,1])
amplif_amplit.cz(0,1)
amplif_amplit.x([0,1])
amplif_amplit.h([0,1])

amplif_amplit.draw()

Mais qiskit permet également de définir l'opérateur de Grover (=oracle + amplification d'amplitude) directement :

In [ ]:
grover_op = grover_operator(oracle)
grover_op.decompose().draw()

On peut alors calculer le nombre de fois qu'il faudrait appliquer l'opérateur pour disposer du résultat optimal

In [ ]:
optimal_num_iterations = math.pi /(4 * math.asin(math.sqrt(1 / 2**grover_op.num_qubits)))-1/2
optimal_num_iterations

#### 1.3. Circuit complet et exécution

Il reste à rajouter la première étape qui consiste à préparer un état de superposition uniforme. 

In [ ]:
qc = QuantumCircuit(grover_op.num_qubits)

qc.h(range(grover_op.num_qubits))
qc.compose(grover_op, inplace=True)
qc.measure_all()

qc.draw()

In [ ]:
#génération d'un pass manager adapté à la machine (ici AerSimulator) et avec un niveau d'optimisation de 1
pass_manager = generate_preset_pass_manager(1, AerSimulator())

#optimisation du circuit au moyen du preset_pass_manager
isa_qc=pass_manager.run(qc)

#execution du job au moyen d'un sampler
sampler=Sampler()
job=sampler.run([isa_qc],shots=1024)
result=job.result()

#représentation sous forme d'histogramme
plot_histogram(result[0].data.meas.get_counts())

### 2. Exercices

<div class="alert alert-block alert-info">
<ul>
    <li>Exécuter l'algorithme précédent sur un modèle incluant du bruit (par exemple le modèle de l'ordinateur ibm_brisbane)
    <li>Conclure sur l'utilisabilité de l'algorithme en pratique, en régime NISQ
</ul>
</div>

In [ ]:
import pickle
from qiskit_aer.noise import NoiseModel

with open('ibm_brisbane_eagle.mdl','rb') as f:
    real_noise_dict = pickle.load(f)

realistic_noise_model = NoiseModel.from_dict(real_noise_dict)


#on execute la même expérience sur ce modele
realistic_sampler = Sampler(options=dict(backend_options=dict(noise_model=realistic_noise_model)))
job=realistic_sampler.run([isa_qc],shots=1024)
result=job.result()
#représentation sous forme d'histogramme
plot_histogram(result[0].data.meas.get_counts())

<div class="alert alert-block alert-info">
A l'aide de l'algorithme de Grover, résoudre le problème de satisfaisabilité suivant :

Un éclairage est constitué de 3 lampes, notées A, B et C.

On cherche à savoir comment allumer les lampes A, B et C pour satisfaire **toutes** les conditions suivantes :
- L'une des trois lampes au moins doit être allumée
- Soit A est allumée, soit B est allumée, soit C est éteinte
- Soit A est éteinte, soit B est allumée, soit C est allumée
- Soit A est éteinte, soit B est allumée, soit C est éteinte
- Soit A est éteinte, soit B est éteinte, soit C est allumée

**Note**: ce problème pourrait très bien être résolu poar la logique simple. Ce n'est pas le but de l'exercice ici.
</div>

In [ ]:
def grover_oracle(marked_states):
    if not isinstance(marked_states, list):
        marked_states = [marked_states]
    # Compute the number of qubits in circuit
    num_qubits = len(marked_states[0])

    qc = QuantumCircuit(num_qubits)
    # Mark each target state in the input list
    for target in marked_states:
        # Flip target bit-string to match Qiskit bit-ordering
        rev_target = target[::-1]
        # Find the indices of all the '0' elements in bit-string
        zero_inds = [ind for ind in range(num_qubits) if rev_target.startswith("0", ind)]
        # Add a multi-controlled Z-gate with pre- and post-applied X-gates (open-controls)
        # where the target bit-string has a '0' entry
        if len(zero_inds)!=0:
            qc.x(zero_inds)
        qc.ccz(0,1,2)
        if len(zero_inds)!=0:
            qc.x(zero_inds)
    return qc

In [ ]:
marked_states = ["111","001","010","100","110","011","101","110","011","010","001"]


oracle = grover_oracle(marked_states)
grover_op = GroverOperator(oracle)
grover_op.decompose().draw()

In [ ]:
optimal_num_iterations = math.floor(math.pi / (4 * math.asin(math.sqrt(1 / 2**grover_op.num_qubits)))-1/2)
optimal_num_iterations

In [ ]:
qc = QuantumCircuit(grover_op.num_qubits)

qc.h(range(grover_op.num_qubits))
qc.compose(grover_op, inplace=True)
qc.measure_all()

qc.draw()

In [ ]:
#génération d'un pass manager adapté à la machine (ici AerSimulator) et avec un niveau d'optimisation de 1
pass_manager = generate_preset_pass_manager(1, AerSimulator())

#optimisation du circuit au moyen du preset_pass_manager
isa_qc=pass_manager.run(qc)

#execution du job au moyen d'un sampler
sampler=Sampler()
job=sampler.run([isa_qc],shots=1024)
result=job.result()

#représentation sous forme d'histogramme
plot_histogram(result[0].data.meas.get_counts())